<h2>Capstone Project_Week 3

<h2>Part 1 of submission

In [411]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<h4>Import data from Wikipedia

In [412]:
import pandas as pd

# The following 3 lines of code code were needed fix an issue with the SSL certificate verification (Mac OS)
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postal_codes = pd.read_html(url)
postal_codes[0].head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h4> Converting the raw data postal_codes into a dataframe df

In [413]:
data=postal_codes[0]
ColumnNames=postal_codes[0].columns.values

df=pd.DataFrame(data, columns=ColumnNames)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h4> Drop rows with Borough=='Not assigned'

In [414]:
df.drop(df.loc[df.Borough=='Not assigned'].index,inplace=True)
df.reset_index(drop=True,inplace=True)
df.head(5)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h4> Determine the shape of dataframe df

In [415]:
print(df.shape)
print('Dataframe df has {} rows and {} columns.'.format(df.shape[0],df.shape[1]))

(103, 3)
Dataframe df has 103 rows and 3 columns.


<h2> Part 2 of submission

<h4> Retrieve coordinates for neighborhoods ...

<h4> ... using geocoder to access coordinates @ Google.

This code did not work. Access to Google data denied?

<h4> ... using <i>pgeocode</i> to access coordinates (uses data from GeoNames geographical database www.geonames.org).

In [416]:
import pgeocode

# Loop through postal codes and retrieve the corresponding coordinates
for postal_code in df.loc[:,'Postal Code']:

    df.loc[df['Postal Code']==postal_code,'latitude_pgeocode'] = pgeocode.Nominatim('CA').query_postal_code(postal_code).latitude
    df.loc[df['Postal Code']==postal_code,'longitude_pgeocode'] = pgeocode.Nominatim('CA').query_postal_code(postal_code).longitude

df.head()

,Postal Code,Borough,Neighbourhood,latitude_pgeocode,longitude_pgeocode
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


<h4> ... using the csv-file provided in the course.

In [417]:
df.loc[:,'latitude']=pd.read_csv(r'https://cocl.us/Geospatial_data').Latitude
df.loc[:,'longitude']=pd.read_csv(r'https://cocl.us/Geospatial_data').Longitude
df.head()

,Postal Code,Borough,Neighbourhood,latitude_pgeocode,longitude_pgeocode,latitude,longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.7276,-79.3148,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,43.773136,-79.239476


<h4> Coordinates from different sources are different. Coordinates latitude_pgeocode/longitude_pgeocode (from library pgeocode) were chosen for further analysis

<h2> Part 3 of submission

<h4> Exploring and clustering the neighborhoods in Toronto

##### Overview of Toronto boroughs and neighbourhoods

In [418]:
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


#### Create a map of Toronto with neighborhood centers superimposed on top.

##### Determine Toronto coordinate to center map

In [419]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


##### Generate unique numerical id for each borough (to enable color coding in map)

In [420]:
df.loc[:,'Borough_No']=None
df.loc[:,'Borough_No']=df.loc[:,'Borough'].replace(df.loc[:,'Borough'].unique(),range(len(df.loc[:,'Borough'].unique())))
df=df.sort_values('Borough_No')
df.reset_index(drop=True,inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood,latitude_pgeocode,longitude_pgeocode,latitude,longitude,Borough_No
0,M3A,North York,Parkwoods,43.7545,-79.3300,43.806686,-79.194353,0
1,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,43.754328,-79.442259,0
2,M2R,North York,"Willowdale, Willowdale West",43.7786,-79.4450,43.709577,-79.445073,0
3,M2P,North York,York Mills West,43.7500,-79.3978,43.662696,-79.400049,0
4,M2J,North York,"Fairview, Henry Farm, Oriole",43.7801,-79.3479,43.761631,-79.520999,0


##### Show map with neighbourhood center points (color coded by borough)

In [505]:
import matplotlib.pylab as pl
from matplotlib import colors

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

df=df.loc[df['latitude_pgeocode'].isna()==False,:] #Drop neighbourhood with missing coordinates (Post Code 'M7R')

# add markers to map
for lat, lng, borough, borough_no, neighborhood in zip(df.latitude_pgeocode, df.longitude_pgeocode, df.Borough, df.Borough_No, df.Neighbourhood):
    color_code=colors.to_hex(pl.cm.tab10(borough_no), keep_alpha=False) # Color code for number of collisions
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color_code,
        fill=True,
        fill_color=color_code,
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
print('\n color coding: boroughs \n')    
map_toronto


 color coding: boroughs 



### Retrieve neighbourhood venues from Foursquare.com

In [422]:
# Foursquare.com credentials and parameters
CLIENT_ID = 'R2B3BSMOAZU414BSOGLAYMB2MJPFUDT1PPEL1M0KZMD4RJFK' # your Foursquare ID
CLIENT_SECRET = '1YM0O41P05UKBSIDUTBL1G4ZYCAKEVRNQ4VFIO40THDU3EWW' # your Foursquare Secret
VERSION = '20201110' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

# Define function to retrieve venues per neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues=getNearbyVenues(df.Neighbourhood, df.latitude_pgeocode, df.longitude_pgeocode, radius=500)
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,Grill Gate,43.753123,-79.451690,Mediterranean Restaurant
3,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,Wolfie's Deli,43.754875,-79.442438,Deli / Bodega
4,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,Tim Hortons,43.754767,-79.443250,Coffee Shop


#### Analyze Each Neighborhood


In [424]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [426]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

#### Let's confirm the new size


In [427]:
toronto_grouped.shape

(96, 261)

#### Let's print each neighborhood along with the top 3 most common venues


In [454]:
num_top_venues = 3

for hood in toronto_grouped['Neighbourhood']:
#    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [455]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 3 venues for each neighborhood.


In [456]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Shanghai Restaurant,Latin American Restaurant,Badminton Court
1,"Alderwood, Long Branch",Pizza Place,Pub,Sandwich Place
2,"Bathurst Manor, Wilson Heights, Downsview North",Mediterranean Restaurant,Fried Chicken Joint,Deli / Bodega
3,Bayview Village,Flower Shop,Park,Trail
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop


<a id='item4'></a>


#### Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 3 clusters.


In [457]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 3 venues for each neighborhood.


In [458]:
# add clustering labels

neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,latitude_pgeocode,longitude_pgeocode,latitude,longitude,Borough_No,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,43.806686,-79.194353,0,1.0,Food & Drink Shop,Park,Dumpling Restaurant
1,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,43.754328,-79.442259,0,0.0,Mediterranean Restaurant,Fried Chicken Joint,Deli / Bodega
2,M2R,North York,"Willowdale, Willowdale West",43.7786,-79.4450,43.709577,-79.445073,0,0.0,Coffee Shop,Gym / Fitness Center,Park
3,M2P,North York,York Mills West,43.7500,-79.3978,43.662696,-79.400049,0,1.0,Park,Convenience Store,Fast Food Restaurant
4,M2J,North York,"Fairview, Henry Farm, Oriole",43.7801,-79.3479,43.761631,-79.520999,0,0.0,Clothing Store,Fast Food Restaurant,Coffee Shop


In [459]:
toronto_merged=toronto_merged.loc[toronto_merged['Cluster Labels'].isna()==False,:]
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

In [460]:
toronto_merged['Cluster Labels'].unique()

array([1, 0, 2])

Visualizing the resulting clusters


In [502]:
# create map
map_clusters = folium.Map(location=[latitude+0.07, longitude+0.011], zoom_start=11.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []

for lat, lon, poi, borough_no, cluster in zip(toronto_merged['latitude_pgeocode'], toronto_merged['longitude_pgeocode'], toronto_merged['Neighbourhood'], df.Borough_No, toronto_merged['Cluster Labels']):
    color_code=colors.to_hex(pl.cm.tab10(borough_no), keep_alpha=False) # Color code for number of collisions
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7.5,
        popup=label,
        color=color_code,
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)

print('\n marker color coding: border color=borough, fill color=cluster\n')
map_clusters


 marker color coding: border color=borough, fill color=cluster



<a id='item5'></a>


#### Examine Clusters


Now, we examine each cluster and determine the discriminating venue categories that distinguish each cluster.


#### Cluster 1


In [462]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,latitude,longitude,Borough_No,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
1,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,0,Mediterranean Restaurant,Fried Chicken Joint,Deli / Bodega
2,North York,"Willowdale, Willowdale West",43.709577,-79.445073,0,0,Coffee Shop,Gym / Fitness Center,Park
4,North York,"Fairview, Henry Farm, Oriole",43.761631,-79.520999,0,0,Clothing Store,Fast Food Restaurant,Coffee Shop
5,North York,"Northwood Park, York University",43.725882,-79.315572,0,0,Massage Studio,Sports Bar,Sandwich Place
6,North York,Downsview,43.647177,-79.381576,0,0,Pizza Place,Coffee Shop,Shopping Mall
7,North York,"Willowdale, Willowdale East",43.640816,-79.381752,0,0,Pizza Place,Ramen Restaurant,Coffee Shop
8,North York,"Humberlea, Emery",43.657952,-79.387383,0,0,Discount Store,Coffee Shop,Grocery Store
9,North York,Bayview Village,43.705369,-79.349372,0,0,Flower Shop,Park,Trail
10,North York,Downsview,43.685347,-79.338106,0,0,Pizza Place,Coffee Shop,Shopping Mall
11,North York,"Bedford Park, Lawrence Manor East",43.651494,-79.375418,0,0,Italian Restaurant,Sandwich Place,Coffee Shop


##### Cluster 1: Lots of coffee shops and Italian food!

#### Cluster 2


In [481]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,latitude,longitude,Borough_No,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,Parkwoods,43.806686,-79.194353,0,1,Food & Drink Shop,Park,Dumpling Restaurant
3,North York,York Mills West,43.662696,-79.400049,0,1,Park,Convenience Store,Fast Food Restaurant
16,North York,Hillcrest Village,43.725900,-79.340923,0,1,Residential Building (Apartment / Condo),Park,Yoga Studio
70,Scarborough,Scarborough Village,43.728496,-79.495697,3,1,Spa,Grocery Store,Park
72,East York,"East Toronto, Broadview North (Old East York)",43.706397,-79.309937,4,1,Park,Convenience Store,Intersection
81,York,Weston,43.696948,-79.411307,5,1,Park,Yoga Studio,Dumpling Restaurant
91,West Toronto,"High Park, The Junction South",43.646435,-79.374846,7,1,Bowling Alley,Park,Residential Building (Apartment / Condo)
94,Central Toronto,Lawrence Park,43.648198,-79.379817,8,1,Photography Studio,Park,Yoga Studio
99,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.628947,-79.394420,8,1,Lawyer,Trail,Park


##### Cluster 2: Lots of places to relax (parks, yoga studios & spa)!

#### Cluster 3


In [497]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + [2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,latitude,longitude,Borough_No,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
65,Scarborough,"Wexford, Maryvale",43.718518,-79.464763,3,2,Auto Garage,Yoga Studio,Fast Food Restaurant


##### Cluster 3: Good place to park your car (auto garages)!